# 데이터 불러오기

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from google.colab import drive
drive.mount('/content/drive')

DATA_PATH = "/content/drive/MyDrive/data/"

SEED = 42 # 시드값

# 데이터 블러오기
train = pd.read_csv(f"{DATA_PATH}titanic_train.csv") # 학습데이터
test = pd.read_csv(f"{DATA_PATH}titanic_test.csv") # 테스트 데이터

# 결측치 처리
age_mean = train["age"].mean()
fare_median = train["fare"].median()
cabin_unk = "UNK"
embarked_mode = train["embarked"].mode()[0]
train["age"] = train["age"].fillna(age_mean)
train["cabin"] = train["cabin"].fillna(cabin_unk)
test["age"] = test["age"].fillna(age_mean)
test["fare"] = test["fare"].fillna(fare_median)
test["cabin"] = test["cabin"].fillna(cabin_unk)
test["embarked"] = test["embarked"].fillna(embarked_mode)

# 특성으로 사용할 변수 선택
cols = ["age","sibsp","parch","fare","pclass","gender","embarked"]
train_ft = train[cols].copy()
test_ft = test[cols].copy()

# 범주형 변수 원핫인코딩
cols = ['gender','embarked']
enc = OneHotEncoder(handle_unknown = 'ignore')
enc.fit(train[cols])
tmp = pd.DataFrame(
    enc.transform(train_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
train_ft = pd.concat([train_ft,tmp],axis=1).drop(columns=cols)
tmp = pd.DataFrame(
    enc.transform(test_ft[cols]).toarray(),
    columns = enc.get_feature_names_out()
)
test_ft = pd.concat([test_ft,tmp],axis=1).drop(columns=cols)

# Min-Max Scaling
scaler = MinMaxScaler()
scaler.fit(train_ft)
train_ft = scaler.transform(train_ft)
test_ft = scaler.transform(test_ft)

# 정답 데이터
target = train["survived"].to_numpy().reshape(-1,1)

train_ft.shape, test_ft.shape, target.shape

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


((916, 10), (393, 10), (916, 1))

In [3]:
target

array([[0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
    

# WorkFlow
1. 데이터 전처리
2. 학습 데이터를 텐서로 변경
3. 인공 신경망 모델 객체 생성
4. 하이퍼파라미터 정리(손실함수 및 옵티마이저 등 선택)
5. 학습 및 테스트 예측하는 loop 구현

# 손실함수

## 회귀문제
  - 정답 데이터 2차원형식으로 생성

In [4]:
import torch

In [8]:
torch.manual_seed(SEED)
# 2개의 샘플 + 3개 피처
x = torch.randn(2,3)

# 2개의 정답 데이터(1차원 형태 시 에러 발생, 2차원형태으로 생성)
y = torch.rand(2,1)

x.shape, y.shape

(torch.Size([2, 3]), torch.Size([2, 1]))

In [9]:
x

tensor([[ 0.3367,  0.1288,  0.2345],
        [ 0.2303, -1.1229, -0.1863]])

In [10]:
y

tensor([[0.8694],
        [0.5677]])

In [11]:
x.shape[1]

3

In [12]:
model = torch.nn.Linear(x.shape[1], 1) # 입력데이터 피처 개수: 3, 출력데이터 예측값 개수: 1
model

Linear(in_features=3, out_features=1, bias=True)

In [13]:
pred = model(x)
pred

tensor([[0.2729],
        [0.1581]], grad_fn=<AddmmBackward0>)

### MSE

In [14]:
# 손실을 계산할 수 있는 손실함수 객체 반환
loss_fn = torch.nn.MSELoss()

# 손실함수 내 첫 번째 인수: 예측값, 두 번째 인수: 정답값 전달
loss = loss_fn(pred, y)
loss

tensor(0.2618, grad_fn=<MseLossBackward0>)

In [15]:
# 역전파
loss.backward

<bound method Tensor.backward of tensor(0.2618, grad_fn=<MseLossBackward0>)>

### MAE

In [27]:
torch.manual_seed(SEED)
# 2개의 샘플 + 3개 피처
x = torch.randn(2,3)

# 2개의 정답 데이터(1차원 형태 시 에러 발생, 2차원형태으로 생성)
y = torch.rand(2,1)

In [28]:
y.dtype

torch.float32

In [22]:
model = torch.nn.Linear(x.shape[1], 1) # 입력데이터 피처 개수: 3, 출력데이터 예측값 개수: 1
model

Linear(in_features=3, out_features=1, bias=True)

In [23]:
pred = model(x)
pred

tensor([[0.2729],
        [0.1581]], grad_fn=<AddmmBackward0>)

In [24]:
loss_fn = torch.nn.L1Loss()
loss = loss_fn(pred, y)
loss

tensor(0.5031, grad_fn=<MeanBackward0>)

In [25]:
# 역전파
loss.backward

<bound method Tensor.backward of tensor(0.5031, grad_fn=<MeanBackward0>)>

## 분류문제

### BCE(Binary Cross Entropy)
- 이진 분류 문제에서 사용되는 손실함수

In [33]:
x

tensor([[ 0.3367,  0.1288,  0.2345],
        [ 0.2303, -1.1229, -0.1863]])

In [38]:
torch.Tensor([0,1])

tensor([0., 1.])

In [34]:
# Tensor > float32
y = torch.Tensor([0,1]).view(-1,1)
y

tensor([[0.],
        [1.]])

In [35]:
y.dtype

torch.float32

In [39]:
x.shape[1]

3

In [40]:
model = torch.nn.Linear(x.shape[1], 1) # 입력 데이터 피처 개수: 3, 출력데이터 예측값 개수: 1

In [41]:
pred = model(x)
pred

tensor([[0.2448],
        [0.2041]], grad_fn=<AddmmBackward0>)

- 정의한 모델의 출력이 시그모이드 함수를 통과한 경우

In [42]:
# 시그모이드 함수르 반환하는 클래스
sig = torch.nn.Sigmoid()
loss_fn = torch.nn.BCELoss()
loss_fn(sig(pred), y)

tensor(0.7097, grad_fn=<BinaryCrossEntropyBackward0>)

- 정의한 모델의 출력이 시그모이드 함수를 통과하지 않은 경우
- 파이토치 권장 > 속도 빠름
- 시그모이드함수 이미 포함되어 있어서 언급 x

In [43]:
loss_fn = torch.nn.BCEWithLogitsLoss()
loss_fn(pred, y)

tensor(0.7097, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

### CE(Multi-class Cross Entropy)
- 다중 분류 문제에서 사용되는 손실함수

In [47]:
# 4개 샘플, 2개 피처
x = torch.randn(4,2)
display(x)
x.shape

tensor([[-0.7658, -0.7506],
        [ 1.3525,  0.6863],
        [-0.3278,  0.7950],
        [ 0.2815,  0.0562]])

torch.Size([4, 2])

In [48]:
x.dtype

torch.float32

In [49]:
x.shape[1]

2

- 다중분류 문제만 정답데이터는 1차원 형태, 데이터타입은 int64 형태

In [50]:
y = torch.tensor([0,1,2,1]) # 1차원 형태
display(y)
y.shape

tensor([0, 1, 2, 1])

torch.Size([4])

In [51]:
y.dtype

torch.int64

In [53]:
x.shape[1]

2

In [54]:
model = torch.nn.Linear(x.shape[1],3) # 입력 데이터 피처 개수: 2, 출력데이터 예측값 개수: 3
pred = model(x)
pred

tensor([[-0.2132, -0.3979, -0.7491],
        [-0.9222, -0.6024, -0.2185],
        [-0.0876, -0.1697,  0.0928],
        [-0.5427, -0.4781, -0.4303]], grad_fn=<AddmmBackward0>)

In [55]:
loss_fn = torch.nn.CrossEntropyLoss()
loss_fn(pred, y)

tensor(1.0237, grad_fn=<NllLossBackward0>)

# 옵티마이터(Optimizer)

- pytorch의 옵티마이저의 주요 파라미터
  - 첫 번째 인수 : 모델 파라미터
  - lr(learning rate) : 학습률
  - weight_decay : 가중치 감소를 적용하는데 사용되는 계수
(defalut = 0)
    - 가중치 감소 하지 않는 경우
L2정규화 지원

In [56]:
model.parameters()

<generator object Module.parameters at 0x7e72af914350>

In [57]:
optim = torch.optim.SGD(model.parameters(), lr = 0.001) # 경사하강법

In [58]:
# 가중치 업데이트
# 역전파 종류 후

# loss.backward
optim.step()

## Adam

In [59]:
optim = torch.optim.Adam(model.parameters())

In [60]:
# 가중치 업데이트
# 역전파 종류 후

# loss.backward
optim.step()

# 입력 데이터 텐서 변환
- 일반적으로 입력데이터 텐서의 dtype은 float32

In [63]:
x_train = torch.Tensor(train_ft) # Tensor > float32

In [64]:
x_train.shape, x_train.dtype

(torch.Size([916, 10]), torch.float32)

# 정답 데이터 텐서 변환

In [65]:
y_train = torch.Tensor(target.reshape(-1,1)) # reshape > 형태 변환

In [66]:
y_train.shape, y_train.dtype

(torch.Size([916, 1]), torch.float32)

In [67]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

# 손실함수 객체 생성

In [69]:
loss_fn = torch.nn.BCEWithLogitsLoss() # 이진분류

# 학습 loop구현
- 에폭 100회

In [70]:
torch.manual_seed(SEED) # 시드값 고정
model = torch.nn.Linear(x_train.shape[1],1).to(device) # 입력데이터 피처 개수: 10, 출력데이터 예측값 개수: 1, gpu에서 계산하기
optimizer = torch.optim.Adam(model.parameters()) # 최적화 > Adam

model.train() # 학습모드 변경
for _ in range(100):
  pred = model(x_train.to(device)) # 예측
  loss = loss_fn(pred, y_train.to(device)) # 손실 계산 후 손실 텐서 반환

  optimizer.zero_grad() # 기울기가 누적되므로, 역전파 실행 전 기울기 0으로 초기화
  loss.backward() # 역전파
  optimizer.step() # 가중치 업데이트

  print(f'epoch loss : {loss.item()}')

epoch loss : 0.6774646639823914
epoch loss : 0.6765610575675964
epoch loss : 0.6756608486175537
epoch loss : 0.6747638583183289
epoch loss : 0.673870325088501
epoch loss : 0.6729802489280701
epoch loss : 0.6720936298370361
epoch loss : 0.6712105870246887
epoch loss : 0.6703311800956726
epoch loss : 0.6694554686546326
epoch loss : 0.6685833930969238
epoch loss : 0.6677150130271912
epoch loss : 0.6668505072593689
epoch loss : 0.6659897565841675
epoch loss : 0.6651329398155212
epoch loss : 0.6642799973487854
epoch loss : 0.6634309887886047
epoch loss : 0.6625858545303345
epoch loss : 0.6617447733879089
epoch loss : 0.6609078049659729
epoch loss : 0.6600747108459473
epoch loss : 0.6592457890510559
epoch loss : 0.6584208607673645
epoch loss : 0.6576001048088074
epoch loss : 0.6567834615707397
epoch loss : 0.6559708714485168
epoch loss : 0.6551624536514282
epoch loss : 0.6543580889701843
epoch loss : 0.653558075428009
epoch loss : 0.6527620553970337
epoch loss : 0.6519702076911926
epoch loss

# 테스트 데이터 구현

In [71]:
x_test = torch.Tensor(test_ft)
x_test.shape, x_test.dtype

(torch.Size([393, 10]), torch.float32)

In [72]:
model.eval() # 평가모드

sig = torch.nn.Sigmoid() # 확률값으로 변환

with torch.no_grad(): # 예측과정에서 경사추적을 할 필요 없기 때문에 with문 안에서 예측 수행, @데코레이터로 활용 가능
  pred = model(x_test.to(device))
  pred = sig(pred) # 음수에서 양수 사이 예측확률 변환
  pred.to('cpu').numpy() # gpu에서 계산된 값을 cpu 이동 후 ndarray변환
pred

tensor([[0.4373],
        [0.4247],
        [0.3782],
        [0.6072],
        [0.6020],
        [0.5011],
        [0.4020],
        [0.4808],
        [0.4819],
        [0.6116],
        [0.3985],
        [0.6072],
        [0.5390],
        [0.5179],
        [0.5790],
        [0.3807],
        [0.4240],
        [0.5048],
        [0.4914],
        [0.3806],
        [0.3820],
        [0.5546],
        [0.6072],
        [0.3779],
        [0.3849],
        [0.5512],
        [0.3949],
        [0.4379],
        [0.4224],
        [0.5259],
        [0.4890],
        [0.4831],
        [0.3644],
        [0.5061],
        [0.4836],
        [0.5231],
        [0.6217],
        [0.4224],
        [0.3795],
        [0.3869],
        [0.5047],
        [0.5915],
        [0.3825],
        [0.3981],
        [0.4820],
        [0.3820],
        [0.4184],
        [0.5088],
        [0.3987],
        [0.5072],
        [0.5479],
        [0.3850],
        [0.4788],
        [0.5053],
        [0.4820],
        [0